In [1]:
import random
import operator

import numpy

from deap import algorithms
from deap import base
from deap import creator
from deap import tools
from deap import gp
import math
import numpy as np
### Graphviz Section ###
import pygraphviz as pgv
import os

if  not "C:\Program Files\Graphviz\\bin" in os.environ["PATH"]: 
    os.environ["PATH"] += os.pathsep + "C:\Program Files\Graphviz\\bin"
print(os.environ["PATH"])

C:\Users\Jan\Storage\Anaconda;C:\Users\Jan\Storage\Anaconda\Library\mingw-w64\bin;C:\Users\Jan\Storage\Anaconda\Library\usr\bin;C:\Users\Jan\Storage\Anaconda\Library\bin;C:\Users\Jan\Storage\Anaconda\Scripts;C:\Users\Jan\Storage\Anaconda\bin;C:\Users\Jan\Storage\Anaconda\condabin;C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v10.0\bin;C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v10.0\libnvvp;C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v11.5\bin;C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v11.5\libnvvp;C:\Program Files\Microsoft MPI\Bin;C:\Windows\system32;C:\Windows;C:\Windows\System32\Wbem;C:\Windows\System32\WindowsPowerShell\v1.0;C:\Windows\System32\OpenSSH;C:\Program Files (x86)\NVIDIA Corporation\PhysX\Common;C:\WINDOWS\system32;C:\WINDOWS;C:\WINDOWS\System32\Wbem;C:\WINDOWS\System32\WindowsPowerShell\v1.0;C:\WINDOWS\System32\OpenSSH;C:\Program Files\Git\cmd;C:\Program Files\NVIDIA Corporation\NVIDIA NvDLISR;C:\Users\Jan\AppData\Local\Microsoft\WindowsApp

In [4]:
def protectedDiv(left, right):
    try:
        if right == 0:
            right += 0.00001
        result = left / right
        return result
    except ZeroDivisionError:
        #print(f"Failed at Division with values {left} {right}")
        return 1

def protectedLog(val):
    try:
        result = math.log(val)
        return result
    except:
        return 1
def logexp(val):
    try:
        return math.exp(val)
    except:
        #print(f"Failed exponential with value {val}")
        return 1

def protectadd(a, b):
    try:
        return a+b
    except:
        return 1

def protectmult(a,b):
    try:
        return a*b
    except:
        return 1

pset = gp.PrimitiveSet("main", 1)
pset.addPrimitive(protectadd, 2)
pset.addPrimitive(protectmult, 2)
pset.addPrimitive(operator.sub, 2)
pset.addPrimitive(protectedDiv, 2)
pset.addPrimitive(math.cos, 1)
pset.addPrimitive(math.sin, 1)
pset.addPrimitive(logexp, 1)
pset.addPrimitive(protectedLog, 1)

pset.renameArguments(ARG0="x")

In [5]:
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", gp.PrimitiveTree, fitness=creator.FitnessMax)

toolbox = base.Toolbox()
toolbox.register("expr", gp.genFull, pset=pset, min_=2, max_=4)
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.expr)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("compile", gp.compile, pset=pset)

data = np.genfromtxt("data.csv", delimiter=",")

def evalFunc(individual):
    func = toolbox.compile(expr=individual)
    total_diff = 0
    for i in range(data.shape[0]):
        total_diff += abs(func(data[i][0]) - data[1])
    return total_diff

toolbox.register("evaluate", evalFunc)
toolbox.register("select", tools.selTournament, tournsize=10)
toolbox.register("mate", gp.cxOnePoint)
toolbox.decorate("mate", gp.staticLimit(key=operator.attrgetter('height'), max_value=50))

def main():
    # random.seed(10)
    pop = toolbox.population(n=1000)
    hof = tools.HallOfFame(1)
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", numpy.mean)
    stats.register("std", numpy.std)
    stats.register("min", numpy.min)
    stats.register("max", numpy.max)

    algorithms.eaSimple(pop, toolbox, 0.7, 0, 50, stats, halloffame=hof)

    return pop, stats, hof

pop, stats, hof = main()
expr = hof.items[0]
nodes, edges, labels = gp.graph(expr)

g = pgv.AGraph()
g.add_nodes_from(nodes)
g.add_edges_from(edges)
g.layout()

for i in nodes:
    n = g.get_node(i)
    n.attr["label"] = labels[i]

g.draw("tree.pdf")

C:\Users\Jan\Storage\Anaconda\lib\site-packages\deap\creator.py:138: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
C:\Users\Jan\Storage\Anaconda\lib\site-packages\deap\creator.py:138: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


gen	nevals	avg        	std        	min    	max        
0  	1000  	9.81617e+43	3.10259e+45	1.62533	9.81617e+46
1  	668   	1.80083e+275	inf        	9.81016	1.80083e+278


C:\Users\Jan\Storage\Anaconda\lib\site-packages\numpy\core\_methods.py:232: RuntimeWarning: overflow encountered in multiply
  x = um.multiply(x, x, out=x)


2  	712   	5.40249e+275	inf        	4.02093	1.80083e+278
3  	676   	3.78069e+293	inf        	19.9419	3.78069e+296
4  	740   	1.50057e+299	inf        	20.9287	1.50001e+302
5  	692   	inf         	inf        	26.272 	1.29239e+308


C:\Users\Jan\Storage\Anaconda\lib\site-packages\numpy\core\_methods.py:178: RuntimeWarning: overflow encountered in reduce
  ret = umr_sum(arr, axis, dtype, out, keepdims, where=where)
C:\Users\Jan\Storage\Anaconda\lib\site-packages\numpy\core\_methods.py:211: RuntimeWarning: overflow encountered in reduce
  arrmean = umr_sum(arr, axis, dtype, keepdims=True, where=where)


6  	738   	inf         	inf        	24.2773	1.29239e+308
7  	680   	inf         	inf        	21.224 	1.38513e+308


C:\Users\Jan\Storage\Anaconda\lib\site-packages\numpy\core\_methods.py:229: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)
<ipython-input-5-acb412fd6791>:16: RuntimeWarning: overflow encountered in add
  total_diff += abs(func(data[i][0]) - data[1])
<ipython-input-4-450429ca2462>:32: RuntimeWarning: overflow encountered in double_scalars
  return a*b
<ipython-input-4-450429ca2462>:5: RuntimeWarning: overflow encountered in double_scalars
  result = left / right


8  	758   	inf         	nan        	21.1425	inf         
9  	728   	inf         	nan        	20.9017	inf         
10 	754   	inf         	nan        	21.3124	inf         


ValueError: math domain error

In [17]:
A = pgv.AGraph()
progs_list = ['neato', 'dot', 'twopi', 'circo', 'fdp', 'nop', 'wc', 'acyclic', 'gvpr',
              'gvcolor', 'ccomps', 'sccmap', 'tred', 'sfdp', 'unflatten']
for prog in progs_list:
    try:
        runprog = A._get_prog(prog)
        print(f'{runprog}')
    except ValueError as e:
        print(f'{prog} gets this error: {str(e).strip()}')

C:\Program Files\Graphviz\bin\neato.exe
C:\Users\Jan\Storage\Anaconda\Scripts\dot.exe
C:\Program Files\Graphviz\bin\twopi.exe
C:\Program Files\Graphviz\bin\circo.exe
C:\Program Files\Graphviz\bin\fdp.exe
C:\Users\Jan\Storage\Anaconda\Scripts\nop.exe
wc gets this error: Program wc not found in path.
C:\Users\Jan\Storage\Anaconda\Scripts\acyclic.exe
gvpr gets this error: Program gvpr not found in path.
C:\Users\Jan\Storage\Anaconda\Scripts\gvcolor.exe
C:\Users\Jan\Storage\Anaconda\Scripts\ccomps.exe
C:\Users\Jan\Storage\Anaconda\Scripts\sccmap.exe
C:\Users\Jan\Storage\Anaconda\Scripts\tred.exe
C:\Program Files\Graphviz\bin\sfdp.exe
C:\Users\Jan\Storage\Anaconda\Scripts\unflatten.exe
